 # Basic Candlestick Pattern Detection
 
 This notebook demonstrates the basic candlestick pattern detection functionalities from the 
 `patternforge.candlestick.patterns` module. We'll cover:
 
 1. Setting up the environment
 2. Loading sample data
 3. Basic single-candle patterns
 4. Two-candle patterns
 5. Three-candle patterns
 6. Visualizing detected patterns


## 1. Setting up the environment

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import yfinance as yf
from datetime import datetime, timedelta
import sys
import os

# Add the parent directory to the path so we can import the patternforge module
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from patternforge.candlestick.patterns import CandlestickPatterns

# Set up plotting
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

## 2. Loading sample data
We'll use Yahoo Finance to get some historical stock data for our analysis.

In [ ]:
# Define date range
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # One year of data

# Download data for Apple
symbol = 'AAPL'
data = yf.download(symbol, start=start_date, end=end_date)

# Display the first few rows
print(f"Downloaded {len(data)} rows of data for {symbol}")
data.head()

Let's create a helper function for visualizing candlestick data and patterns

In [ ]:
def plot_candlestick_with_patterns(df, patterns_dict, title='Candlestick Chart with Patterns', figsize=(15, 10)):
    """
    Plot candlestick chart with pattern markers
    
    Args:
        df: DataFrame with OHLC data
        patterns_dict: Dictionary with pattern name as key and boolean Series as value
        title: Chart title
        figsize: Figure size
    """
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot candlestick
    width = 0.6
    width2 = width * 0.8
    
    up = df[df.Close >= df.Open]
    down = df[df.Close < df.Open]
    
    # Plot up candles
    ax.bar(up.index, up.Close-up.Open, width=width2, bottom=up.Open, color='green', alpha=0.5)
    ax.bar(up.index, up.High-up.Close, width=0.1, bottom=up.Close, color='green', alpha=0.5)
    ax.bar(up.index, up.Open-up.Low, width=0.1, bottom=up.Low, color='green', alpha=0.5)
    
    # Plot down candles
    ax.bar(down.index, down.Open-down.Close, width=width2, bottom=down.Close, color='red', alpha=0.5)
    ax.bar(down.index, down.High-down.Open, width=0.1, bottom=down.Open, color='red', alpha=0.5)
    ax.bar(down.index, down.Close-down.Low, width=0.1, bottom=down.Low, color='red', alpha=0.5)
    
    # Plot patterns
    colors = plt.cm.tab10.colors
    marker_y_positions = df['High'] * 1.01  # Just above the highest point
    pattern_found = False
    
    for i, (pattern_name, pattern_series) in enumerate(patterns_dict.items()):
        if pattern_series is None or isinstance(pattern_series, tuple):
            continue
            
        pattern_indices = df.index[pattern_series]
        if len(pattern_indices) > 0:
            pattern_found = True
            ax.scatter(pattern_indices, marker_y_positions.loc[pattern_indices], 
                      marker='^', color=colors[i % len(colors)], s=100, label=pattern_name)
    
    # Add bullish/bearish pattern tuples
    for i, (pattern_name, pattern_tuple) in enumerate(patterns_dict.items()):
        if not isinstance(pattern_tuple, tuple):
            continue
            
        bullish, bearish = pattern_tuple
        bullish_indices = df.index[bullish]
        bearish_indices = df.index[bearish]
        
        if len(bullish_indices) > 0:
            pattern_found = True
            ax.scatter(bullish_indices, marker_y_positions.loc[bullish_indices], 
                      marker='^', color=colors[i % len(colors)], s=100, label=f"Bullish {pattern_name}")
            
        if len(bearish_indices) > 0:
            pattern_found = True
            ax.scatter(bearish_indices, marker_y_positions.loc[bearish_indices], 
                      marker='v', color=colors[(i+1) % len(colors)], s=100, label=f"Bearish {pattern_name}")
    
    # Format the chart
    ax.set_title(title, fontsize=16)
    ax.set_ylabel('Price', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    # Format x-axis with dates
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    
    # Add volume as a subplot
    if 'Volume' in df.columns:
        volume_ax = ax.twinx()
        volume_ax.bar(df.index, df.Volume, width=width, alpha=0.3, color='gray')
        volume_ax.set_ylabel('Volume', fontsize=14)
        volume_ax.set_ylim(0, df.Volume.max() * 4)  # Set upper limit to avoid overlap
    
    # Show legend only if patterns were found
    if pattern_found:
        ax.legend(loc='upper left')
    
    plt.tight_layout()
    return fig, ax
